# Flowers Image Classification with TensorFlow on Cloud ML Engine

This notebook demonstrates how to do image classification from scratch on a flowers dataset using the Estimator API.

In [1]:
import os
PROJECT = "qwiklabs-gcp-02-80d4be27465a" # REPLACE WITH YOUR PROJECT ID
BUCKET = "qwiklabs-gcp-02-80d4be27465a" # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE = "cnn"

# do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["MODEL_TYPE"] = MODEL_TYPE
os.environ["TFVERSION"] = "1.13"  # Tensorflow version

In [2]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Input functions to read JPEG images

The key difference between this notebook and [the MNIST one](./mnist_models.ipynb) is in the input function.
In the input function here, we are doing the following:
* Reading JPEG images, rather than 2D integer arrays.
* Reading in batches of batch_size images rather than slicing our in-memory structure to be batch_size images.
* Resizing the images to the expected HEIGHT, WIDTH. Because this is a real-world dataset, the images are of different sizes. We need to preprocess the data to, at the very least, resize them to constant size.

## Run as a Python module

Since we want to run our code on Cloud ML Engine, we've packaged it as a python module.

The `model.py` and `task.py` containing the model code is in <a href="flowersmodel">flowersmodel</a>

**Complete the TODOs in `model.py` before proceeding!**

Once you've completed the TODOs, run it locally for a few steps to test the code.

In [7]:
%%bash
rm -rf flowersmodel.tar.gz flowers_trained
gcloud ai-platform local train \
    --module-name=flowersmodel.task \
    --package-path=${PWD}/flowersmodel \
    -- \
    --output_dir=${PWD}/flowers_trained \
    --train_steps=5 \
    --learning_rate=0.01 \
    --batch_size=2 \
    --model=$MODEL_TYPE \
    --augment \
    --train_data_path=gs://cloud-ml-data/img/flower_photos/train_set.csv \
    --eval_data_path=gs://cloud-ml-data/img/flower_photos/eval_set.csv




INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--output_dir=/home/jupyter/training-data-analyst/courses/machine_learning/deepdive/08_image/labs/flowers_trained', u'--train_steps=5', u'--learning_rate=0.01', u'--batch_size=2', u'--model=cnn', u'--augment', u'--train_data_path=gs://cloud-ml-data/img/flower_photos/train_set.csv', u'--eval_data_path=gs://cloud-ml-data/img/flower_photos/eval_set.csv'], u'job_name': u'flowersmodel.task'}, u'task': {}}
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 300, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff897f88cd0>, '_model_dir': '/home/jupyter/training-data-analyst/courses/machine_learning/deepdive/08_image/labs/flowers_trained/', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 

Now, let's do it on ML Engine. Note the --model parameter

In [8]:
%%bash
OUTDIR=gs://${BUCKET}/flowers/trained_${MODEL_TYPE}
JOBNAME=flowers_${MODEL_TYPE}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=flowersmodel.task \
    --package-path=${PWD}/flowersmodel \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://$BUCKET \
    --scale-tier=BASIC_GPU \
    --runtime-version=$TFVERSION \
    -- \
    --output_dir=$OUTDIR \
    --train_steps=1000 \
    --learning_rate=0.01 \
    --batch_size=40 \
    --model=$MODEL_TYPE \
    --augment \
    --batch_norm \
    --train_data_path=gs://cloud-ml-data/img/flower_photos/train_set.csv \
    --eval_data_path=gs://cloud-ml-data/img/flower_photos/eval_set.csv

gs://qwiklabs-gcp-02-80d4be27465a/flowers/trained_cnn us-central1 flowers_cnn_191230_040131
jobId: flowers_cnn_191230_040131
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [flowers_cnn_191230_040131] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe flowers_cnn_191230_040131

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs flowers_cnn_191230_040131


## Monitor training with TensorBoard

To activate TensorBoard within the JupyterLab UI navigate to "<b>File</b>" - "<b>New Launcher</b>".   Then double-click the 'Tensorboard' icon on the bottom row.

TensorBoard 1 will appear in the new tab.  Navigate through the three tabs to see the active TensorBoard.   The 'Graphs' and 'Projector' tabs offer very interesting information including the ability to replay the tests.

You may close the TensorBoard tab when you are finished exploring.

## Deploying and predicting with model

Deploy the model:

In [ ]:
%%bash
MODEL_NAME="flowers"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/flowers/trained_${MODEL_TYPE}/export/exporter | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ai-platform versions delete --quiet ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ai-platform models delete ${MODEL_NAME}
gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
gcloud ai-platform versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

To predict with the model, let's take one of the example images that is available on Google Cloud Storage <img src="http://storage.googleapis.com/cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg" />

The online prediction service expects images to be base64 encoded as described [here](https://cloud.google.com/ml-engine/docs/tensorflow/online-predict#binary_data_in_prediction_input).

In [ ]:
%%bash
IMAGE_URL=gs://cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg

# Copy the image to local disk.
gsutil cp $IMAGE_URL flower.jpg

# Base64 encode and create request message in json format.
python -c 'import base64, sys, json; img = base64.b64encode(open("flower.jpg", "rb").read()).decode(); print(json.dumps({"image_bytes":{"b64": img}}))' &> request.json

Send it to the prediction service

In [ ]:
%%bash
gcloud ai-platform predict \
    --model=flowers \
    --version=${MODEL_TYPE} \
    --json-instances=./request.json

<pre>
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>

model.py

#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(v = tf.logging.INFO)

LIST_OF_LABELS = "daisy,dandelion,roses,sunflowers,tulips".split(',')
HEIGHT = 299
WIDTH = 299
NUM_CHANNELS = 3
NCLASSES = 5

def linear_model(img, mode, hparams):
    X = tf.reshape(tensor = img, shape = [-1,HEIGHT * WIDTH]) #flatten
    ylogits = tf.layers.dense(input = X, units = NCLASSES, activation = None)
    return ylogits, NCLASSES

def dnn_model(img, mode, hparams):
    X = tf.reshape(tensor = img, shape = [-1, HEIGHT * WIDTH]) #flatten
    h1 = tf.layers.dense(input = X, units = 300, activation = tf.nn.relu)
    h2 = tf.layers.dense(input = h1, units = 100, activation = tf.nn.relu)
    h3 = tf.layers.dense(input = h2, units = 30, activation = tf.nn.relu)
    ylogits = tf.layers.dense(input = h3, units = NCLASSES, activation = None)
    return ylogits, NCLASSES

def dnn_dropout_model(img, mode, hparams):
    dprob = hparams.get("dprob", 0.1)

    X = tf.reshape(tensor = img, shape = [-1, HEIGHT * WIDTH]) #flatten
    h1 = tf.layers.dense(input = X, units = 300, activation = tf.nn.relu)
    h2 = tf.layers.dense(input = h1, units = 100, activation = tf.nn.relu)
    h3 = tf.layers.dense(input = h2, units = 30, activation = tf.nn.relu)
    h3d = tf.layers.dropout(inputs = h3, rate = dprob, training = (mode == tf.estimator.ModeKeys.TRAIN)) #only dropout when training
    ylogits = tf.layers.dense(input = h3d, units = NCLASSES, activation = None)
    return ylogits, NCLASSES

def cnn_model(img, mode, hparams):
    ksize1 = hparams.get("ksize1", 5)
    ksize2 = hparams.get("ksize2", 5)
    nfil1 = hparams.get("nfil1", 10)
    nfil2 = hparams.get("nfil2", 20)
    dprob = hparams.get("dprob", 0.25)

    c1 = tf.layers.conv2d(inputs = img, filters = nfil1,
                          kernel_size = ksize1, strides = 1,
                          padding = "same", activation = tf.nn.relu) # shape = (batch_size, HEIGHT, WIDTH, nfil1)
    
    p1 = tf.layers.max_pooling2d(inputs = c1, pool_size = 2, strides = 2) # shape = (batch_size, HEIGHT // 2, WIDTH // 2, nfil1)
    
    c2 = tf.layers.conv2d(inputs = p1, filters = nfil2,
                          kernel_size = ksize2, strides = 1, 
                          padding = "same", activation = tf.nn.relu) # shape = (batch_size, HEIGHT // 2, WIDTH // 2, nfil2)
    
    p2 = tf.layers.max_pooling2d(inputs = c2, pool_size = 2, strides = 2) # shape = (batch_size, HEIGHT // 4, WIDTH // 4, nfil2)

    outlen = p2.shape[1] * p2.shape[2] * p2.shape[3] # HEIGHT // 4 * WIDTH // 4 * nfil2
    p2flat = tf.reshape(tensor = p2, shape = [-1, outlen]) # shape = (batch_size, HEIGHT // 4 * WIDTH // 4 * nfil2)

    # Apply batch normalization
    if hparams["batch_norm"]:
        h3 = tf.layers.dense(inputs = p2flat, units = 300, activation = None)
        h3 = tf.layers.batch_normalization(inputs = h3, training = (mode == tf.estimator.ModeKeys.TRAIN)) # only batchnorm when training
        h3 = tf.nn.relu(features = h3)
    else:  
        h3 = tf.layers.dense(inputs = p2flat, units = 300, activation = tf.nn.relu)
  
    # Apply dropout
    h3d = tf.layers.dropout(inputs = h3, rate = dprob, training = (mode == tf.estimator.ModeKeys.TRAIN))

    ylogits = tf.layers.dense(inputs = h3d, units = NCLASSES, activation = None)
  
    # Apply batch normalization once more
    if hparams["batch_norm"]:
        ylogits = tf.layers.batch_normalization(inputs = ylogits, training = (mode == tf.estimator.ModeKeys.TRAIN))

    return ylogits, NCLASSES

def read_and_preprocess_with_augment(image_bytes, label = None):
    return read_and_preprocess(image_bytes, label, augment = True)
    
def read_and_preprocess(image_bytes, label = None, augment = False):
    # Decode the image, end up with pixel values that are in the -1, 1 range    
    image = tf.image.decode_jpeg(image_bytes, channels=NUM_CHANNELS) #TODO: decode contents into JPEG
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)    #TODO: convert JPEG tensor to floats between 0 and 1
    image = tf.expand_dims(input = image, axis = 0) # resize_bilinear needs batches
    
    if augment:
        #TODO: Add image augmentation functions
        image=tf.image.resize_bilinear(image, [HEIGHT+10, WIDTH+10], align_corners=False)
        image=tf.squeeze(image)
        image=tf.image.random_crop(image, [HEIGHT,WIDTH,NUM_CHANNELS])
        image=tf.image.random_brightness(image,max_delta=63.0/255)
        image=tf.image.random_contrast(image,lower=0.2,upper=1.8)
    else:
        image = tf.image.resize_bilinear(images = image, size = [HEIGHT, WIDTH], align_corners = False)
        image = tf.squeeze(input = image, axis = 0) # remove batch dimension
        
    # Pixel values are in range [0,1], convert to [-1,1]
    image = tf.subtract(x = image, y = 0.5)
    image = tf.multiply(x = image, y = 2.0)
    return {"image": image}, label

def serving_input_fn():
    # Note: only handles one image at a time 
    feature_placeholders = {"image_bytes": tf.placeholder(dtype = tf.string, shape = [])}
    image, _ = read_and_preprocess(tf.squeeze(input = feature_placeholders["image_bytes"]))
    image["image"] = tf.expand_dims(image["image"], axis = 0)
    return tf.estimator.export.ServingInputReceiver(features = image, receiver_tensors = feature_placeholders)

def make_input_fn(csv_of_filenames, batch_size, mode, augment=False):
    def _input_fn(): 
        def decode_csv(csv_row):
            filename, label = tf.decode_csv(records = csv_row, record_defaults = [[""],[""]])
            image_bytes = tf.read_file(filename = filename)
            return image_bytes, label
        
        # Create tf.data.dataset from filename
        dataset = tf.data.TextLineDataset(filenames = csv_of_filenames).map(map_func = decode_csv)
        
        if augment:
            dataset = dataset.map(read_and_preprocess_with_augment)    #TODO: map read_and_preprocess_with_augment
        else:
            dataset = dataset.map(read_and_preprocess)                  #TODO: map read_and_preprocess

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this
 
        dataset = dataset.repeat(count = num_epochs).batch(batch_size = batch_size)
        return dataset.make_one_shot_iterator().get_next()
    return _input_fn
    
def image_classifier(features, labels, mode, params):
    model_functions = {
        "linear": linear_model,
        "dnn": dnn_model,
        "dnn_dropout": dnn_dropout_model,
        "cnn": cnn_model}
    model_function = model_functions[params["model"]] 
    ylogits, nclasses = model_function(features["image"], mode, params)

    probabilities = tf.nn.softmax(logits = ylogits)
    class_int = tf.cast(x = tf.argmax(input = ylogits, axis = 1), dtype = tf.uint8)
    class_str = tf.gather(params = LIST_OF_LABELS, indices = tf.cast(x = class_int, dtype = tf.int32))
  
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        # Convert string label to int
        labels_table = tf.contrib.lookup.index_table_from_tensor(mapping = tf.constant(value = LIST_OF_LABELS, dtype = tf.string))
        labels = labels_table.lookup(keys = labels)

        loss = tf.reduce_mean(input_tensor = tf.nn.softmax_cross_entropy_with_logits_v2(logits = ylogits, labels = tf.one_hot(indices = labels, depth = NCLASSES)))
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            # This is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(key = tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(control_inputs = update_ops):
                train_op = tf.contrib.layers.optimize_loss(
                    loss = loss, 
                    global_step = tf.train.get_global_step(),
                    learning_rate = params["learning_rate"],
                    optimizer = "Adam")
            eval_metric_ops = None
        else:
            train_op = None
            eval_metric_ops =  {"accuracy": tf.metrics.accuracy(labels = labels, predictions = class_int)}
    else:
        loss = None
        train_op = None
        eval_metric_ops = None
 
    return tf.estimator.EstimatorSpec(
        mode = mode,
        predictions = {"probabilities": probabilities, 
                       "classid": class_int, 
                       "class": class_str},
        loss = loss,
        train_op = train_op,
        eval_metric_ops = eval_metric_ops,
        export_outputs = {"classes": tf.estimator.export.PredictOutput(
            {"probabilities": probabilities, 
             "classid": class_int, 
             "class": class_str})}
    )

def train_and_evaluate(output_dir, hparams):
    tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
    
    EVAL_INTERVAL = 300 # every 5 minutes
    
    # Instantiate base estimator class for custom model function
    estimator = tf.estimator.Estimator(
        model_fn = image_classifier,
        params = hparams,
        config = tf.estimator.RunConfig(
            save_checkpoints_secs = EVAL_INTERVAL),
            model_dir = output_dir)
    
    # Set estimator"s train_spec to use train_input_fn and train for so many steps
    train_spec = tf.estimator.TrainSpec(
        input_fn = make_input_fn(
            hparams["train_data_path"],
            hparams["batch_size"],
            mode = tf.estimator.ModeKeys.TRAIN,
            augment = hparams["augment"]),
        max_steps = hparams["train_steps"])

    # Create exporter that uses serving_input_fn to create saved_model for serving
    exporter = tf.estimator.LatestExporter(
        name = "exporter", 
        serving_input_receiver_fn = serving_input_fn)

    # Set estimator"s eval_spec to use eval_input_fn and export saved_model
    eval_spec = tf.estimator.EvalSpec(
        input_fn = make_input_fn(
            hparams["eval_data_path"],
            hparams["batch_size"],
            mode = tf.estimator.ModeKeys.EVAL),
        steps = None,
        exporters = exporter,
        start_delay_secs = EVAL_INTERVAL,
        throttle_secs = EVAL_INTERVAL)

    # Run train_and_evaluate loop
    tf.estimator.train_and_evaluate(
        estimator = estimator, 
        train_spec = train_spec, 
        eval_spec = eval_spec)